## 1. Load Raw Data

In [5]:
import pandas as pd
import numpy as np
import re

df_gdp = pd.read_csv("../../data/raw/GDP-countries.csv", skiprows=4)

print("Preview of raw GDP dataset:")
display(df_gdp.head())

Preview of raw GDP dataset:


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.420993e+10,2.496326e+10,2.707802e+10,3.177483e+10,3.028492e+10,3.381219e+10,...,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12,1.133818e+12,1.205974e+12,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.715223e+10,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.190511e+10,1.270803e+10,1.363092e+10,1.446926e+10,1.580394e+10,1.692124e+10,...,7.000282e+11,6.940513e+11,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11,8.147285e+11,6.700257e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11,8.487516e+10,8.039694e+10,NaN


## 2. Select Relevant Year Columns

In [6]:
year_cols = [str(y) for y in range(2000, 2022 + 1)]

gdp_filtered = df_gdp[["Country Name", "Country Code"] + year_cols]

print("Filtered GDP dataset with selected year columns:")
display(gdp_filtered.head())

Filtered GDP dataset with selected year columns:


,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,1.873453e+09,1.896457e+09,1.961844e+09,2.044112e+09,2.254831e+09,2.360017e+09,2.469783e+09,2.677641e+09,...,2.727850e+09,2.790850e+09,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09
1,Africa Eastern and Southern,AFE,2.872017e+11,2.609922e+11,2.678150e+11,3.557164e+11,4.426962e+11,5.166611e+11,5.802408e+11,6.655987e+11,...,9.624413e+11,9.787438e+11,8.983089e+11,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12
2,Afghanistan,AFG,3.521418e+09,2.813572e+09,3.825701e+09,4.520947e+09,5.224897e+09,6.203257e+09,6.971758e+09,9.747886e+09,...,2.014642e+10,2.049713e+10,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10
3,Africa Western and Central,AFW,1.421401e+11,1.500585e+11,1.793901e+11,2.077550e+11,2.585667e+11,3.170965e+11,4.027242e+11,4.715378e+11,...,8.442026e+11,9.039337e+11,7.780221e+11,7.000282e+11,6.940513e+11,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11
4,Angola,AGO,9.129595e+09,8.936079e+09,1.528559e+10,1.781270e+10,2.355206e+10,3.697090e+10,5.238103e+10,6.526642e+10,...,1.323391e+11,1.359668e+11,9.049642e+10,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11


## 3. Analyze Row-wise Missing Values

In [7]:
row_wise_missing = gdp_filtered.isnull().sum(axis=1)

print("Row-wise analysis of missing values (count of missing years per country):")
print(row_wise_missing.value_counts().sort_index())

countries_with_missing = gdp_filtered[row_wise_missing > 0]
missing_counts = row_wise_missing[row_wise_missing > 0]

result = pd.DataFrame({
    "Country Name": countries_with_missing["Country Name"],
    "Missing Values Count": missing_counts
})

print("\nCountries with missing values and their respective counts:")
display(result.reset_index(drop=True))


Row-wise analysis of missing values (count of missing years per country):
0     247
1       2
2       5
4       1
6       1
8       2
9       1
11      1
15      1
19      1
23      4
Name: count, dtype: int64

Countries with missing values and their respective counts:


,Country Name,Missing Values Count
0,American Samoa,2
1,Channel Islands,1
2,Cuba,2
3,Cayman Islands,6
4,Eritrea,11
5,Gibraltar,23
6,Guam,2
7,Not classified,23
8,St. Martin (French part),19
9,Northern Mariana Islands,2


## 4. Drop Rows with Too Many Missing Values

In [8]:
gdp_filtered_cleaned = gdp_filtered[row_wise_missing <= 10]

print("DataFrame after dropping rows with more than 10 missing values:")
display(gdp_filtered_cleaned.head(10))

# Recalculate missing values for the cleaned data
cleaned_row_wise_missing = gdp_filtered_cleaned.isnull().sum(axis=1)

print("\nRow-wise analysis of missing values in the cleaned DataFrame:")
print(cleaned_row_wise_missing.value_counts().sort_index())


DataFrame after dropping rows with more than 10 missing values:


,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,1.873453e+09,1.896457e+09,1.961844e+09,2.044112e+09,2.254831e+09,2.360017e+09,2.469783e+09,2.677641e+09,...,2.727850e+09,2.790850e+09,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09
1,Africa Eastern and Southern,AFE,2.872017e+11,2.609922e+11,2.678150e+11,3.557164e+11,4.426962e+11,5.166611e+11,5.802408e+11,6.655987e+11,...,9.624413e+11,9.787438e+11,8.983089e+11,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12
2,Afghanistan,AFG,3.521418e+09,2.813572e+09,3.825701e+09,4.520947e+09,5.224897e+09,6.203257e+09,6.971758e+09,9.747886e+09,...,2.014642e+10,2.049713e+10,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10
3,Africa Western and Central,AFW,1.421401e+11,1.500585e+11,1.793901e+11,2.077550e+11,2.585667e+11,3.170965e+11,4.027242e+11,4.715378e+11,...,8.442026e+11,9.039337e+11,7.780221e+11,7.000282e+11,6.940513e+11,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11
4,Angola,AGO,9.129595e+09,8.936079e+09,1.528559e+10,1.781270e+10,2.355206e+10,3.697090e+10,5.238103e+10,6.526642e+10,...,1.323391e+11,1.359668e+11,9.049642e+10,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11
5,Albania,ALB,3.584570e+09,4.059064e+09,4.515003e+09,5.801712e+09,7.406646e+09,8.256658e+09,9.150528e+09,1.111694e+10,...,1.279699e+10,1.329632e+10,1.147017e+10,1.198867e+10,1.325827e+10,1.537951e+10,1.558511e+10,1.524146e+10,1.803201e+10,1.901724e+10
6,Andorra,AND,1.432606e+09,1.548266e+09,1.764280e+09,2.366942e+09,2.900245e+09,3.161084e+09,3.459338e+09,3.957625e+09,...,3.193513e+09,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.324648e+09,3.380613e+09
7,Arab World,ARB,7.634116e+11,7.447285e+11,7.466493e+11,8.300765e+11,9.873160e+11,1.210080e+12,1.439471e+12,1.674641e+12,...,2.874276e+12,2.926205e+12,2.568631e+12,2.524103e+12,2.598475e+12,2.907330e+12,2.949355e+12,2.596423e+12,3.077907e+12,3.738584e+12
8,United Arab Emirates,ARE,1.043374e+11,1.033116e+11,1.098162e+11,1.243464e+11,1.478244e+11,1.806175e+11,2.221165e+11,2.579161e+11,...,4.002185e+11,4.141054e+11,3.702755e+11,3.692553e+11,3.905168e+11,4.270494e+11,4.179897e+11,3.494730e+11,4.151788e+11,5.027319e+11
9,Argentina,ARG,2.842038e+11,2.686968e+11,9.772400e+10,1.275870e+11,1.646579e+11,1.987371e+11,2.325573e+11,2.875305e+11,...,5.520251e+11,5.263197e+11,5.947493e+11,5.575323e+11,6.436284e+11,5.248199e+11,4.477547e+11,3.857405e+11,4.865641e+11,6.327901e+11



Row-wise analysis of missing values in the cleaned DataFrame:
0    247
1      2
2      5
4      1
6      1
8      2
9      1
Name: count, dtype: int64


## 5. Interpolate Missing Year Values

In [9]:
gdp_filled = gdp_filtered_cleaned.copy()

# Year columns start from the third column
year_columns = gdp_filled.columns[2:]

# Interpolate across years (columns are transposed, interpolated, then transposed back)
gdp_filled[year_columns] = (
    gdp_filled[year_columns]
    .T
    .interpolate(method="linear", limit_direction="both")
    .T
)

print("DataFrame after linear interpolation of missing year values:")
display(gdp_filled.head())


DataFrame after linear interpolation of missing year values:


,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,1.873453e+09,1.896457e+09,1.961844e+09,2.044112e+09,2.254831e+09,2.360017e+09,2.469783e+09,2.677641e+09,...,2.727850e+09,2.790850e+09,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09
1,Africa Eastern and Southern,AFE,2.872017e+11,2.609922e+11,2.678150e+11,3.557164e+11,4.426962e+11,5.166611e+11,5.802408e+11,6.655987e+11,...,9.624413e+11,9.787438e+11,8.983089e+11,8.289612e+11,9.730251e+11,1.012291e+12,1.009747e+12,9.334072e+11,1.085605e+12,1.191639e+12
2,Afghanistan,AFG,3.521418e+09,2.813572e+09,3.825701e+09,4.520947e+09,5.224897e+09,6.203257e+09,6.971758e+09,9.747886e+09,...,2.014642e+10,2.049713e+10,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10
3,Africa Western and Central,AFW,1.421401e+11,1.500585e+11,1.793901e+11,2.077550e+11,2.585667e+11,3.170965e+11,4.027242e+11,4.715378e+11,...,8.442026e+11,9.039337e+11,7.780221e+11,7.000282e+11,6.940513e+11,7.778404e+11,8.332889e+11,7.972952e+11,8.581145e+11,8.936399e+11
4,Angola,AGO,9.129595e+09,8.936079e+09,1.528559e+10,1.781270e+10,2.355206e+10,3.697090e+10,5.238103e+10,6.526642e+10,...,1.323391e+11,1.359668e+11,9.049642e+10,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11


## 6. Scale GDP Values to Billions

In [11]:
gdp_filled[year_columns] = (gdp_filled[year_columns] / 1e9).round(4)

print("DataFrame after scaling GDP values to billions:")
display(gdp_filled.head())

DataFrame after scaling GDP values to billions:


,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,1.8735,1.8965,1.9618,2.0441,2.2548,2.3600,2.4698,2.6776,...,2.7278,2.7908,2.9629,2.9836,3.0924,3.2762,3.3958,2.4819,2.9294,3.2793
1,Africa Eastern and Southern,AFE,287.2017,260.9922,267.8150,355.7164,442.6962,516.6611,580.2408,665.5987,...,962.4413,978.7438,898.3089,828.9612,973.0251,1012.2912,1009.7465,933.4072,1085.6049,1191.6386
2,Afghanistan,AFG,3.5214,2.8136,3.8257,4.5209,5.2249,6.2033,6.9718,9.7479,...,20.1464,20.4971,19.1342,18.1166,18.7535,18.0532,18.7994,19.9559,14.2600,14.4972
3,Africa Western and Central,AFW,142.1401,150.0585,179.3901,207.7550,258.5667,317.0965,402.7242,471.5378,...,844.2026,903.9337,778.0221,700.0282,694.0513,777.8404,833.2889,797.2952,858.1145,893.6399
4,Angola,AGO,9.1296,8.9361,15.2856,17.8127,23.5521,36.9709,52.3810,65.2664,...,132.3391,135.9668,90.4964,52.7616,73.6902,79.4507,70.8980,48.5016,66.5051,104.3997


## 7. Verify Missing Values After Imputation

In [12]:
final_row_wise_missing = gdp_filled.isnull().sum(axis=1)

print("Row-wise analysis of missing values in the final DataFrame:")
print(final_row_wise_missing.value_counts().sort_index())


Row-wise analysis of missing values in the final DataFrame:
0    259
Name: count, dtype: int64


## 8. Final Preview and Save Cleaned Data

In [13]:
print("Final preview of cleaned GDP dataset:")
display(gdp_filled.head())

output_file_path = "../../data/processed/cleaned-GDP-countries.csv"
gdp_filled.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")

Final preview of cleaned GDP dataset:


,Country Name,Country Code,2000,2001,2002,2003,2004,2005,2006,2007,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,1.8735,1.8965,1.9618,2.0441,2.2548,2.3600,2.4698,2.6776,...,2.7278,2.7908,2.9629,2.9836,3.0924,3.2762,3.3958,2.4819,2.9294,3.2793
1,Africa Eastern and Southern,AFE,287.2017,260.9922,267.8150,355.7164,442.6962,516.6611,580.2408,665.5987,...,962.4413,978.7438,898.3089,828.9612,973.0251,1012.2912,1009.7465,933.4072,1085.6049,1191.6386
2,Afghanistan,AFG,3.5214,2.8136,3.8257,4.5209,5.2249,6.2033,6.9718,9.7479,...,20.1464,20.4971,19.1342,18.1166,18.7535,18.0532,18.7994,19.9559,14.2600,14.4972
3,Africa Western and Central,AFW,142.1401,150.0585,179.3901,207.7550,258.5667,317.0965,402.7242,471.5378,...,844.2026,903.9337,778.0221,700.0282,694.0513,777.8404,833.2889,797.2952,858.1145,893.6399
4,Angola,AGO,9.1296,8.9361,15.2856,17.8127,23.5521,36.9709,52.3810,65.2664,...,132.3391,135.9668,90.4964,52.7616,73.6902,79.4507,70.8980,48.5016,66.5051,104.3997


Cleaned data saved to ../../data/processed/cleaned-GDP-countries.csv
